In [58]:
import pandas as pd

df1 = pd.read_excel('data-penumpang-bus-transjakarta-tahun-2021-(1764692849646).xlsx')
df1.head()

,periode_data,jenis,kode_trayek,trayek,jumlah_penumpang
0,202101,Angkutan Umum Integrasi,3E,Puri Kembangan - Sentraland Cengkareng,24385
1,202101,Angkutan Umum Integrasi,1R,Senen - Tanah Abang,13518
2,202101,Angkutan Umum Integrasi,1Q,Rempoa - Blok M,29449
3,202101,Angkutan Umum Integrasi,1P,Senen - Bundaran Senayan,8793
4,202101,Angkutan Umum Integrasi,1N,Tanah Abang - Blok M,9987


In [59]:
df1.columns

Index(['periode_data', 'jenis', 'kode_trayek', 'trayek', 'jumlah_penumpang'], dtype='object')

In [60]:
df1['periode_data'].head()

0    202101
1    202101
2    202101
3    202101
4    202101
Name: periode_data, dtype: int64

In [61]:
df1[['jenis']].value_counts()

jenis                  
Mikrotrans                 848
Angkutan Umum Integrasi    469
BRT                        156
Name: count, dtype: int64

In [62]:
df1_BRT = df1[df1['jenis'] == 'BRT'].copy()
df1_Angkutan_Umum_integrasi = df1[df1['jenis'] == 'Angkutan Umum Integrasi'].copy()
df1_Mikrotrans = df1[df1['jenis'] == 'Mikrotrans'].copy()

In [63]:
pd.set_option('display.max_rows', None)
df1_BRT_penumpang = df1_BRT[['periode_data', 'kode_trayek', 'jumlah_penumpang']].sort_values(by=['kode_trayek', 'periode_data']).head()

In [ ]:
import calendar

# Extract year and month from periode_data
df1_BRT['year'] = df1_BRT['periode_data'] // 100
df1_BRT['month'] = df1_BRT['periode_data'] % 100

# Get days in month
df1_BRT['days_in_month'] = df1_BRT.apply(
    lambda row: calendar.monthrange(row['year'], row['month'])[1], 
    axis=1
)

# Calculate daily passengers
df1_BRT['jumlah_penumpang_per_day'] = df1_BRT['jumlah_penumpang'] / df1_BRT['days_in_month']

pd.set_option('display.max_rows', None)
df1_BRT[['periode_data', 'kode_trayek', 'jumlah_penumpang', 'jumlah_penumpang_per_day']].sort_values(by=['periode_data', 'kode_trayek'])



,periode_data,kode_trayek,jumlah_penumpang,jumlah_penumpang_per_day
234,202101,1,773717,24958.612903
233,202101,10,356211,11490.677419
232,202101,11,134632,4342.967742
231,202101,12,137793,4444.935484
230,202101,13,266290,8590.000000
229,202101,2,323751,10443.580645
228,202101,3,444971,14353.903226
227,202101,4,249575,8050.806452
226,202101,5,400573,12921.709677
225,202101,6,320411,10335.838710


In [65]:
df1_Angkutan_Umum_integrasi[['kode_trayek']].value_counts()

kode_trayek
10K            12
11D            12
11Q            12
12B            12
1B             12
1F             12
1E             12
1C             12
1M             12
1N             12
1P             12
1H             12
1Q             12
4F             12
5B             12
1R             12
8K             12
9D             12
9E             12
4B             12
6H             12
6C             12
5M             12
5F             12
6M             12
8D             12
8C             12
7P             12
7D             12
7B             12
7A             12
6R             12
8E             12
9H             11
3E             10
6N             10
GR4             8
GR5             8
1A              8
1A              4
2Q              3
2P              3
3E              3
12A             1
6Q              1
6F              1
GR3             1
GR2             1
Name: count, dtype: int64

In [66]:
df1_Mikrotrans[['kode_trayek']].value_counts()

kode_trayek
JAK,19         12
JAK.01         12
JAK.06         12
JAK.02         12
JAK.03         12
JAK.04         12
JAK.07         12
JAK.05         12
JAK.09         12
JAK.08         12
JAK.56         12
JAK.58         12
JAK.10         12
JAK.11         12
JAK.117        12
JAK.112        12
JAK.13         12
JAK.12         12
JAK.16         12
JAK.17         12
JAK.14         12
JAK.15         12
JAK.18         12
JAK.20         12
JAK.22         12
JAK.21         12
JAK.30         12
JAK.31         12
JAK.24         12
JAK.25         12
JAK.26         12
JAK.27         12
JAK.28         12
JAK.29         12
JAK.34         12
JAK.35         12
JAK.32         12
JAK.33         12
JAK.36         12
JAK.37         12
JAK.39         12
JAK.38         12
JAK.73         12
JAK.74         12
JAK.40         12
JAK.41         12
JAK.42         12
JAK.43         12
JAK.44         12
JAK.45         12
JAK.46         12
JAK.47         12
JAK.49         12
JAK.50         12
JAK.51         1